# Create predictions using the BERTje finetuned model

This notebook loads our BERTje finetuned model and creates predictions.

In [ ]:
# Install dependencies
!pip uninstall accelerate transformers -y
!pip install -U accelerate>=0.21.0
!pip3 install transformers
!pip3 install datasets
!pip3 install pandas
!pip3 install torch
!pip3 install scikit-learn
!pip3 install numpy
!pip3 install nltk emoji==0.6.0

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=6500916cc51ae68879fc387f7b2e36bd09af547b7993a60b13f846ecada10cd9
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
# Create the tokenizer
from transformers import AutoTokenizer

bertje_tokenizer = AutoTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Load and prepare the data
import pandas as pd

raw_data = pd.read_csv('test_data_text_final.csv')

# Normalize the data for bertje
data_norm_bertje = raw_data

X_bertje = data_norm_bertje['text'].tolist()

In [ ]:
# SKIP if not using google drive/colab
# Load the models from google drive

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Uni-23-24/Machine\ Learning\ Project/Final\ Assignment/models/


Mounted at /content/gdrive
/content/gdrive/My Drive/Uni-23-24/Machine Learning Project/Final Assignment/models


In [ ]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F

def batch_predictions(data, model, model_tokenizer, batch_size=8):

  n_samples = len(data)
  res = []

  model.eval()

  for i in tqdm(range(0, n_samples, batch_size)):

    # torch.no_grad() is only used as a performance improvement here
    with torch.no_grad():
      batch_data = data[i:i+batch_size]
      batch_tok = model_tokenizer(batch_data, padding=True, truncation=True, return_tensors='pt')
      batch_res = model(**batch_tok)

      # Turn the predictions into the classes
      probs = F.softmax(batch_res['logits'], dim=1)
      pred_classes = torch.argmax(probs, dim=1)

      res.extend([x.item() for x in pred_classes])

  return res

In [ ]:
# Save output as specified by the gold file format
def save_predictions(predictions, path):
  df = pd.DataFrame({'id': [i+1 for i in range(len(predictions))], 'abusive_offensive_not': predictions})
  df.to_csv(path, index=False)

### Evaluate BERTje fine-tuned on the DEV dataset

In [ ]:
from transformers import AutoModelForSequenceClassification
# Load the bertje model
bertje_model_path = '/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertje_finetuned' # change to your local models path
bertje_model = AutoModelForSequenceClassification.from_pretrained(bertje_model_path, use_safetensors=True, num_labels=3)

In [ ]:
# Make the predictions in batches
bertje_pred = batch_predictions(X_bertje, bertje_model, bertje_tokenizer)
print(bertje_pred[:10])

100%|██████████| 409/409 [02:13<00:00,  3.05it/s]

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [ ]:
# Turn the classes into labels
bertje_class_to_label = {0: "ABUSIVE", 1: "NOT", 2: "OFFENSIVE"}
bertje_pred_labels = [bertje_class_to_label[class_] for class_ in bertje_pred]
print(bertje_pred_labels[:10])

['NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'OFFENSIVE', 'NOT', 'NOT', 'NOT', 'NOT']


In [ ]:
# Save the predictions
save_predictions(bertje_pred_labels, '/content/bertje_finetuned_final_pred.csv')